# SSMEO Lab 01 : Image coordinates, distortion model

***

## Part 1 : Coordinate projection

In [ ]:
# Load packages 

import numpy as np
import cv2
from scipy.optimize import fsolve
from parse_xml import parse_xml
import matplotlib.pyplot as plt

# Set path to the data folder downloaded on moodle
path = ''

img_id = 1092311568

im_path = path + f'{img_id}_marked.jpg'

img = cv2.imread(im_path)

cam_IO_path = path + f'cam_param.txt'

out_path = 'path to save your outputs'

# Your code : load camera IO coefficients from cam_param.txt file and assign to variables (h, w, c, cx, cy, k1, k2, p1, p2)


In [ ]:
marker_path = f'path_to/YOUR_FILE_NAME_HERE.xml'

### 1.1 Load Agisoft manual measurements

In [ ]:
# Call the parse_xml function to load and parse your data in a np.array with ids and a second np.array with uv coordinates
id, uv_raw = parse_xml(marker_path)

print(f"Loaded {uv_raw.shape[0]} measurements :")
for i in range(uv_raw.shape[0]):
    print(f"Id : {id[i]}, u : {uv_raw[i,0]}, v : {uv_raw[i,1]}")

### 1.2 Definition of conversion between image coordinate systems

Up to now, measurements were expressed in "Top Left" image coordinates, expressed in pixels :
    <ul>
    <li>The origin is the top-left corner of the image</li>
    <li>U is oriented horizontally →</li>
    <li>V is oriented downward ↓</li>
    </ul> 
    
In the next part of this lab, it is necessary to express measurements in perspective centered coordinates, express with unitless coordinates :
    <ul>
    <li>The origin is the perspective center of the image, the center of the pixels array shifted along x and y axis by the principal point offset in pixels $c_x, c_y$ </li>
    <li>X is oriented horizontally →</li>
    <li>Y is oriented downward ↓</li>
    </ul> 

Given : 
    <ul>
    <li>$(u,v)$ : coordinates in top-left image coordinate [px]</li>
    <li>$(x,y)$ : coordinates in perspective-centered coordinates [-]</li>
    <li>$c_x, c_y$ : perspective center offset [px]</li>
    <li>$w, h$ : image width, height [px]</li>
    <li>$c$ : focal length [px]</li>
    </ul> 
    
\begin{equation*}
x = \frac{u - (\frac{w-1}{2} + c_x)}{c}
\tag{1}
\end{equation*}


\begin{equation*}
y = \frac{v - (\frac{h-1}{2} + c_y)}{c}
\tag{2}
\end{equation*}

<div class="alert alert-block alert-info">
Note that to convert to perspective sensor coordinates, one aspect of the camera intrinsic orientation is considered : the center offset.<br>

This offset is generally due to the imperfect mounting of the lens with respect to the sensor, causing the center of the lens to be slightly shifted with respect to the center of the sensor.
</div>

(a) Define function **topleft2perspective** to project pixel coordinates from sensor to perspective centered frame

(b) Define function **perspective2topleft** to project pixel coordinates from perspective to sensor centered frame

In [ ]:
def topleft2perspective(uv, h, w, cx, cy, c):
    '''
    Project from top left to perspective image coordinate 
    
    Parameters
    ----------
    uv : np.array(Nx2), the array of measurements to project
        
    h, w : image height and width
         
    cx, cy, c : corresponding camera calibration parameters
            
    Return
    ------
    xy : np.array(Nx2), the reprojected array 
    '''
    # Your code : convert coordinates

    return xy

#Define function to project from sensor to perspective coordinate system
def perspective2topleft(xy, h, w, cx, cy, c):
    '''
    Project from perspective to top-left image coordinate 
    
    Parameters
    ----------
    xy : np.array(Nx2), the array of measurements to project

    h, w : image height and width

    cx, cy, c : corresponding camera calibration parameters
    
    Return
    ------
    uv : np.array(Nx2), the reprojected array    
    '''
    # Your code : convert coordinates
    
    return uv

In [ ]:
#Test that you reprojections are correct :
topleft_dummy = np.array([[0,0],
                          [(w-1)/2+cx, (h-1)/2+cy]])

perspective_dummy = np.array(([[0,0],
                               [(-(w-1)/2-cx)/c, (-(h-1)/2-cy)/c]]))

#Test the results of your conversion function
assert(np.isclose(topleft2perspective(topleft_dummy, h, w, cx, cy, c),
                  [[(-(w-1)/2-cx)/c, (-(h-1)/2-cy)/c],[0, 0]],
                  atol = 1e-3).all())
assert(np.isclose(perspective2topleft(perspective_dummy, h, w, cx, cy, c),
                  [[((w-1)/2+cx), (h-1)/2+cy],[0, 0]],
                 atol = 1e-3).all())

### 1.3 Project and save measurements

In [ ]:
#Convert uv (top left) to xy (perspective)
xy_raw = 

print(f"Measurements in perspective coordinates :")
for i in range(xy_raw.shape[0]):
    print(f"Id = {id[i]} : x = {xy_raw[i,0]:.8f}, y = {xy_raw[i,1]:.8f}")

 #### Save your measurements along with ids. 
 
Appart from the **id_xy.txt** and **id_uv.txt** measurements file, screenshots of each of your measurements will be exported to the **/measurements** folder.  

You can compare the close range screenshots of your measurements with the provided one (folder raw_data/GCPs_1092311568/) to assess the precision of your marking

In [ ]:
#Concatenate id with to both uv and xy
id_uv_raw = np.concatenate((np.array(id).astype(int).reshape(-1,1),uv_raw),axis=1)
id_xy_raw = np.concatenate((np.array(id).astype(int).reshape(-1,1),xy_raw),axis=1)

# Save those into two separate files
np.savetxt(out_path + f'{img_id}_id_uv_raw.txt',
           id_uv_raw, fmt='%f',
           header = 'id; u; v',
           delimiter = ';')

np.savetxt(out_path + f'{img_id}_id_xy_raw.txt',
           id_xy_raw, fmt='%f',
           header = 'id; x; y',
           delimiter = ';')

print(f"Saving {xy_raw.shape[0]} measurements to {out_path}{img_id}_id_xy_raw.txt and {out_path}{img_id}_id_uv_raw.txt :")

for i in range(uv_raw.shape[0]):
    
    uv_raw = uv_raw.astype(int)
    blue = (200,200,40)
    orn = (0,120,255)
    
    cv2.line(img, (uv_raw[i,0], uv_raw[i,1]-25), (uv_raw[i,0], uv_raw[i,1]+25), blue, 1)
    cv2.line(img, (uv_raw[i,0]-25, uv_raw[i,1]), (uv_raw[i,0]+25, uv_raw[i,1]), blue, 1)
    
    cv2.rectangle(img, (uv_raw[i,0]-350, uv_raw[i,1]-350), (uv_raw[i,0]+350, uv_raw[i,1]+350), blue, 50)
    cv2.rectangle(img, (uv_raw[i][0]-50, uv_raw[i][1]-50), (uv_raw[i][0]+50, uv_raw[i][1]+50), orn, 5)
    cv2.putText(img, str(id[i]), (uv_raw[i,0]-60, uv_raw[i,1]-65), cv2.FONT_HERSHEY_SIMPLEX, 1, orn, 4, cv2.LINE_AA)

    cv2.imwrite(out_path + f"Manual_GCP_{id[i]}_{img_id}.jpg",
                img[uv_raw[i,1]-350:uv_raw[i,1]+350,uv_raw[i,0]-350:uv_raw[i,0]+350])

## Part 2 : Undistort manual measurements
***

<div class="alert alert-block alert-info">
<b>Objectives:</b> Cameras are not perfect and distortion due to imperfect mounting and lens distortion must be corrected to accurately map each pixel. 
    
In this section you will implement a simple distortion model and visualize its effect on your measurements
</div>

### 2.0 Set up



In [ ]:
#Load raw measurements you did in Part 1.:

id_xy_raw = np.loadtxt(path +f'measurements/{img_id}_id_xy_raw.txt', delimiter=';')
print(f"Loaded {id_xy_raw.shape[0]} measurements :")
print(id_xy_raw)  

### 2.1 Define camera distortion model

We will use the simplified Contrady-Brown distortion model. 
This model relates distorded image coordinates $x', y'$ (the points you just measured), to the undistorded ones $x, y$ through radial coefs ($k_1, k_2$) and tangential coefs ($p_1, p_2$) :
    



\begin{equation*}
x(1 + k_1\cdot r² + k_2 \cdot r⁴) + p_1(r² + 2\cdot x²) + 2\cdot p_2\cdot x\cdot y - x' = 0
\tag{1}
\end{equation*}

\begin{equation*}
y(1 + k_1\cdot r² + k_2\cdot r⁴) + p_2(r² + 2\cdot y²) + 2\cdot p_1\cdot x\cdot y - y' = 0
\tag{2}
\end{equation*}

With $r² = x² + y²$      


#### (a) Define model

We will use scipy non-linear solver to find for any measurment on the distorded image ($x', y'$) the corresponding undistorded measurement ($x, y$).

Define **undisort** function, which returns the two symbolic equations (1) and (2), given model's parameters, and undistorded coordinate of a measurement

In [ ]:
def undistort(var,x_d,y_d,k1,k2,p1,p2):  
    '''
    Define equation (1) and (2) given a complet set of parameter and distorded measurement coordinates
    
    Parameters
    ----------
    var : tupple containing x and y variables. This are the two variables scipy solver will solve for
        
    x_d, y_d : floats, the coordinates of a measurment on the distorded image
        
    k1, k2, p1, p2 : floats, Contrady-Brown distortion model coefficients
        
    Return
    ------
    
    list of size 2 containing both eqation of the Contrady-Brown model
    '''
    #Unpack variables
    x, y = var
    #Your code : 2 elements list containing equation (1) and (2) depending on x,y

    return

Have a look at [scipy.optimize.fsolve documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html) to understand how to format the input.  
Initial guess can be initialized with the raw measurements. Hence your raw measurements will be called twice, once packed in **var** tupple, as initial guess for the fsolve to start optimizing, and once in x_d and y_d variables, representing the raw coordinate in the **undistort** function

In [ ]:
# Your code : call the fsolve method on the undistort function to solve for x and y at each measurement


# Your code : store into xy_corrected Nx2 np.array


#### (b) Visualize the effect of the distortion on your measurements and auto evaluate your implementation

The **plot_measurements** function provided below will do this for you (no need to modify it)


In [ ]:
def plot_measurements(img, raw_pts, undist_pts):
    raw_pts_i = raw_pts.astype(int)
    undist_pts_i = undist_pts.astype(int)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    r = (255,0,0)
    g = (0,255,0)
    for i in range(len(raw_pts)):
        cv2.line(img, (raw_pts_i[i]+[-15, 0]), (raw_pts_i[i]+[ 15, 0]), r, 1)
        cv2.line(img, (raw_pts_i[i]+[0, -15]), (raw_pts_i[i]+[0,  15]), r, 1)
        cv2.line(img, (undist_pts_i[i]+[-15, 0]), (undist_pts_i[i]+[ 15, 0]), g, 1)
        cv2.line(img, (undist_pts_i[i]+[0, -15]), (undist_pts_i[i]+[0,  15]), g, 1)
        plt.imshow(img[raw_pts_i[i,1]-150:raw_pts_i[i,1]+150,
                        raw_pts_i[i,0]-150:raw_pts_i[i,0]+150])
        plt.show()


The image provided contains indication for you to evaluate your measurements and their distortion.  
For each GCP, plot_measurement will display a zoom around it. 

On each image, the red square represents the area where your raw measurement should land. The red cross represents the actual position of your raw manual measurement.
If your measurement does not fall into the square, GCP manual measurements or coordinate conversion might be incorrect.

The green square represents the area where your undistorted measurement should land. The green cross represents the actual position of your undistorted measurement.
If your measurement does not fall into the square, you  mast likely have an issue with the distortion part.


In [ ]:
# Your code : project your corrections to top left coordinates and display 
uv_corrected =

#Load the original measurements in top left coordinates
uv_raw = np.loadtxt(out_path + f'{img_id}_id_uv_raw.txt', delimiter=';')[:,1:]

im_path = f'raw_data/{img_id}_assessment.jpg'
img = cv2.imread(im_path)

In [ ]:
plot_measurements(img, uv_raw, uv_corrected)

In [ ]:
# Your code : concatenate measurements ids, u and v into a Nx3 np.array
# and save it to a txt file, same format as part 1
id_uv_corrected =

np.savetxt(out_path + f'{img_id}_id_uv_corrected.txt',
           id_uv_corrected,
           fmt='%f',
           header = 'id; u corrected; v corrected',
           delimiter = ';')

print('Measurements displacement from undistortion [px]: ')
print(uv_raw-uv_corrected)

## Part 3 : Visualize the camera distortion model 



<div class="alert alert-block alert-info">
<b>Objectives:</b> In this section, we aim at understanding the effect of the different distortion coefficient on the image correction.
</div>

The function **grid_points** generates a regular square grid on the image we spacing of **k** pixels. 

Use it to generate a grid every 100 pixel and undistort those points using the same function as in **Part 2**.

Try using different distortion coefficients to understand the influence of each one of them on the total distortion.


In [ ]:
def grid_points(img, k):
    '''
    Generates a regular grid of points from an image in top left coordinates
    
    Parameters
    ----------
    img : cv2.image
        the image from which take a grid
    k : int
        the grid spacing in pixel
            
    Return
    ------
    grid : Nx2 np.array
        the array of the grid, each line contains x,y coordinate of a given node
    '''
    height, width, channels = img.shape
    
    x, y = np.meshgrid(np.arange(0,width, k),
                       np.arange(0,height, k))
    return np.array([x.flatten(), y.flatten()]).T

In [ ]:
#Your code : Generate the grid with one point every 100 pixels and project into perspective centered coordinates

In [ ]:
# Define arbitrary distortion coefficient to oberve the effect of each component on the distortion
k1_dummy = 0 # 0.1
k2_dummy = 0 # 0.1
p1_dummy = 0 # 0.05
p2_dummy = 0.05 # 0.05

In [ ]:
#Your code : Apply distortion model to the grid. 

Visualize the result of the undistorded coordinates estimation

In [ ]:
def plot_distortions(img, raw_pts, undist_pts, thickness):
    dist = np.linalg.norm(raw_pts - undist_pts, axis=1)
    min_d = np.min(dist)
    max_d = np.max(dist)

    raw_pts_i = raw_pts.astype(int)
    undist_pts_i = undist_pts.astype(int)

    for i in range(len(raw_pts)):
        color = (0,
                 255*(1 -(dist[i]-min_d)/(max_d-min_d))**2,
                 255*((dist[i]-min_d)/(max_d-min_d))**0.5)
        
        cv2.line(img, (raw_pts_i[i]), (undist_pts_i[i]), color, thickness)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(20,20))
    plt.imshow(img)

In [ ]:
#Load image & plot
im_path = f'raw_data/{img_id}.jpg'
img = cv2.imread(im_path)

In [ ]:
#Your code : project both grids (raw and corrected) to sensor centered coordinates and visualize the results
# Use a thickness of 15 to easily observe distortion rays 

In [ ]:
print(grid_uv-grid_uv_corrected)